In [1]:
!pip install mlflow boto3 awscli optuna lightgbm imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/8

In [2]:
import os, getpass

os.environ["AWS_ACCESS_KEY_ID"] = getpass.getpass("Enter AWS Access Key ID: ")
os.environ["AWS_SECRET_ACCESS_KEY"] = getpass.getpass("Enter AWS Secret Access Key: ")
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

Enter AWS Access Key ID: ··········
Enter AWS Secret Access Key: ··········


In [3]:
import boto3
s3 = boto3.client('s3')
print(s3.list_buckets())

{'ResponseMetadata': {'RequestId': 'BKGFGWK71FSDN1Z5', 'HostId': 'PL40ytj8zzAmJ9hPUiboNmnTUTTiFnna3CcIxd4vYB2OheDJu/FOsP3sUpaDO37kB3bL/9DnD0A=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'PL40ytj8zzAmJ9hPUiboNmnTUTTiFnna3CcIxd4vYB2OheDJu/FOsP3sUpaDO37kB3bL/9DnD0A=', 'x-amz-request-id': 'BKGFGWK71FSDN1Z5', 'date': 'Fri, 17 Oct 2025 09:02:17 GMT', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Buckets': [{'Name': 'project1-mlflow-bucket', 'CreationDate': datetime.datetime(2025, 10, 12, 12, 4, 13, tzinfo=tzlocal())}], 'Owner': {'DisplayName': 'adityasahusomu02', 'ID': '866466d6c4d8a9893e39cdce3c468d75133f23de2c907deeda4f1a78a7ca565b'}}


In [39]:
import mlflow
mlflow.set_tracking_uri("http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/")

print("Tracking URI:", mlflow.get_tracking_uri())

Tracking URI: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/


In [40]:
mlflow.set_experiment("exp-11-knn_with HP tuning")

<Experiment: artifact_location='s3://project1-mlflow-bucket/866522987519852827', creation_time=1760674060870, experiment_id='866522987519852827', last_update_time=1760674060870, lifecycle_stage='active', name='exp-11-knn_with HP tuning', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [41]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna

In [42]:
url = "https://raw.githubusercontent.com/adityasahusomu/Youtube_Comment_Analyzer/refs/heads/main/cleaned_reddit_dataset.csv"
df = pd.read_csv(url).dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [43]:
df = df.dropna(subset=['category'])

ngram_range = (1,3)
max_feature = 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_feature)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X,y)

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
      mlflow.set_tag('mlflow.runName', f"{model_name}_SMOTE_TFIDF_Trigrams")
      mlflow.set_tag("experiment_type", "algorithm_comparision")

      mlflow.log_param("algo_name", model_name)

      model.fit(X_train, y_train)
      y_pred = model.predict(X_test)

      accuracy = accuracy_score(y_test, y_pred)
      mlflow.log_metric("accuracy", accuracy)

      classification_rep = classification_report(y_test, y_pred, output_dict =True)

      for label, metrics in classification_rep.items():
        if isinstance(metrics, dict):
          for metric, value in metrics.items():
            mlflow.log_metric(f"{label}_{metric}", value)

      mlflow.sklearn.log_model(model, f"{model_name}_model")

# Optuna (Bayesian Optimization)
  # Optuna Obj func
def objective_knn(trial):
    n_neighbors = trial.suggest_int("n_neighbors", 3, 30)
    p = trial.suggest_categorical('p', [1,2])      # 1-> Manhattan, 2-> Euclidean
    model = KNeighborsClassifier(n_neighbors=n_neighbors, p=p)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))

  #Optuna running on knn

def run_optuna_experiment():
    study = optuna.create_study(direction ='maximize')
    study.optimize(objective_knn, n_trials=30)

    best_params = study.best_params
    best_model = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'], p=best_params['p'])

    log_mlflow('KNN', best_model, X_train, X_test, y_train, y_test)

run_optuna_experiment()

# In this case, the model is trained 31 times [30 trials + one inside log_mlflow()]





[I 2025-10-17 11:31:50,550] A new study created in memory with name: no-name-f5280bb4-e955-4f97-9665-cc99ba59dfbc
[I 2025-10-17 11:31:58,984] Trial 0 finished with value: 0.5740858169520185 and parameters: {'n_neighbors': 8, 'p': 2}. Best is trial 0 with value: 0.5740858169520185.
[I 2025-10-17 11:32:24,581] Trial 1 finished with value: 0.3904037201437328 and parameters: {'n_neighbors': 10, 'p': 1}. Best is trial 0 with value: 0.5740858169520185.
[I 2025-10-17 11:32:50,224] Trial 2 finished with value: 0.39611075882477276 and parameters: {'n_neighbors': 6, 'p': 1}. Best is trial 0 with value: 0.5740858169520185.
[I 2025-10-17 11:32:59,284] Trial 3 finished with value: 0.5633058549989431 and parameters: {'n_neighbors': 11, 'p': 2}. Best is trial 0 with value: 0.5740858169520185.
[I 2025-10-17 11:33:24,957] Trial 4 finished with value: 0.37814415556964703 and parameters: {'n_neighbors': 24, 'p': 1}. Best is trial 0 with value: 0.5740858169520185.
[I 2025-10-17 11:33:34,007] Trial 5 finis

🏃 View run KNN_SMOTE_TFIDF_Trigrams at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/866522987519852827/runs/47b1f91861274e13b1d034617740d73a
🧪 View experiment at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/866522987519852827
